In [1]:
# basics
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
from collections import Counter
# import copy
import seaborn as sns
from time import time
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

# sklearn
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold

# pyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.modules.loss import _WeightedLoss

# stratified kfold
!pip install /kaggle/input/iterative-stratification/iterative-stratification-master/
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

Processing /kaggle/input/iterative-stratification/iterative-stratification-master
  Created wheel for iterative-stratification: filename=iterative_stratification-0.1.6-py3-none-any.whl size=8401 sha256=49869fdbc6f5a3b39988b66fadf657caf6f2f4e1b381a40eb20196f0cef7bd9d
  Stored in directory: /root/.cache/pip/wheels/b8/47/3f/eb4af42d124f37d23d6f13a4c8bbc32c1d70140e6e1cecb4aa
Successfully built iterative-stratification


In [2]:
# load raw data
data_dir = '../input/lish-moa/'
train_features = pd.read_csv(data_dir + 'train_features.csv')
train_targets_scored = pd.read_csv(data_dir + 'train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(data_dir + 'train_targets_nonscored.csv')
train_drug = pd.read_csv(data_dir + 'train_drug.csv')
test_features = pd.read_csv(data_dir + 'test_features.csv')
sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')

train_features_extra = pd.read_csv('../input/moa-fe-extra-data/MoA_FE_Extra_Data.csv')

# keep only nonscored targets with >0 positive labels
keep_list = (np.where(train_targets_nonscored.iloc[:,1:].values.sum(axis=0)>0)[0] + 1).tolist()
train_targets_nonscored = train_targets_nonscored.iloc[:,[0]+keep_list]

print('train_features: {}'.format(train_features.shape))
print('train_targets_scored: {}'.format(train_targets_scored.shape))
print('train_targets_nonscored: {}'.format(train_targets_nonscored.shape))
print('train_drug: {}'.format(train_drug.shape))
print('test_features: {}'.format(test_features.shape))
print('fe_extra_features: {}'.format(fe_extra_features.shape))
print('sample_submission: {}'.format(sample_submission.shape))

train_features: (23814, 876)
train_targets_scored: (23814, 207)
train_targets_nonscored: (23814, 332)
train_drug: (23814, 2)
test_features: (3982, 876)


NameError: name 'fe_extra_features' is not defined

In [3]:
# function to set single seed for everything
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)

    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# set seed to 42
seed_everything(42)

# Feature Engineering

In [4]:
# define g/c features
g_col = [col for col in train_features.columns if col.startswith('g-')]
c_col = [col for col in train_features.columns if col.startswith('c-')]

In [5]:
def feature_engineering(x_train, x_test, seed):
    
    # set seed
    seed_everything(seed)

    # sig_id & cp_type
    sig_train = x_train[['sig_id', 'cp_type']]
    sig_test = x_test[['sig_id', 'cp_type']]
    print('sig_id & cp_type', sig_train.shape, sig_test.shape)
    
    # OHE for cp_time, cp_dose
    cp_dose = {'D1': 0, 'D2': 1}
    cp_time = {24:0, 48:1, 72:2}
    cp_dose_train = x_train.cp_dose.map(cp_dose)
    cp_time_train = x_train.cp_time.map(cp_time)
    cp_dose_test = x_test.cp_dose.map(cp_dose)
    cp_time_test = x_test.cp_time.map(cp_time)
    cp_train = pd.DataFrame({'cp_dose':cp_dose_train, 'cp_time':cp_time_train}).reset_index(drop=True)
    cp_test = pd.DataFrame({'cp_dose':cp_dose_test, 'cp_time':cp_time_test}).reset_index(drop=True)
    print('OHE for cp_time, cp_dose', cp_train.shape, cp_test.shape)
    
    # RankGauss scaling
    n_quantiles = 100
    qt = QuantileTransformer(n_quantiles=n_quantiles, random_state=seed, output_distribution='normal').fit(x_train[g_col + c_col])
    rg_train = pd.DataFrame(qt.transform(x_train[g_col + c_col]), columns=['rg_' + col for col in g_col + c_col]).reset_index(drop=True)
    rg_test = pd.DataFrame(qt.transform(x_test[g_col + c_col]), columns=['rg_' + col for col in g_col + c_col]).reset_index(drop=True)
    print('RankGauss scaling', rg_train.shape, rg_test.shape)
    print(rg_train.head())
    
    # PCA for g_col
    g_n_comp = 600
    rg_g_col = [col for col in rg_train.columns if col.startswith('rg_g')]
    pca = PCA(n_components=g_n_comp, random_state=seed).fit(rg_train[rg_g_col])
    pca_g_train = pd.DataFrame(pca.transform(rg_train[rg_g_col]), columns=['pca_g-' + str(i) for i in range(g_n_comp)]).reset_index(drop=True)
    pca_g_test = pd.DataFrame(pca.transform(rg_test[rg_g_col]), columns=['pca_g-' + str(i) for i in range(g_n_comp)]).reset_index(drop=True)
    print('PCA for g_col', pca_g_train.shape, pca_g_test.shape)
    print(pca_g_train.head())

    # PCA for c_col
    c_n_comp = 50
    rg_c_col = [col for col in rg_train.columns if col.startswith('rg_c')]
    pca = PCA(n_components=c_n_comp, random_state=seed).fit(rg_train[rg_c_col])
    pca_c_train = pd.DataFrame(pca.transform(rg_train[rg_c_col]), columns=['pca_c-' + str(i) for i in range(c_n_comp)]).reset_index(drop=True)
    pca_c_test = pd.DataFrame(pca.transform(rg_test[rg_c_col]), columns=['pca_c-' + str(i) for i in range(c_n_comp)]).reset_index(drop=True)
    print('PCA for c_col', pca_c_train.shape, pca_c_test.shape)
    print(pca_c_train.head())

    # combine g & c PCA features
    pca_gc_train = pd.concat((pca_g_train, pca_c_train),axis=1).reset_index(drop=True)
    pca_gc_test  =pd.concat((pca_g_test, pca_c_test),axis=1).reset_index(drop=True)
    print('combine g & c PCA features', pca_gc_train.shape, pca_gc_test.shape)
    print(pca_gc_train.head())

    # FS by Variance Threshold
    thsld = 0.85
    data_train = pd.concat([rg_train, pca_gc_train], axis=1)
    data_test = pd.concat([rg_test, pca_gc_test], axis=1)
    variance_threshold = VarianceThreshold(thsld).fit(data_train)
    selected_features = variance_threshold.get_support(True).tolist()
    fs_train = data_train.iloc[:,selected_features].reset_index(drop=True)
    fs_test = data_test.iloc[:,selected_features].reset_index(drop=True)
    print('FS by Variance Threshold', fs_train.shape, fs_test.shape)
    print(fs_train.head())

    # function to cluster raw g/c
    n_clusters_g = 22
    n_clusters_c = 4
    def gc_cluster(train, test, n_clusters_g, n_clusters_c, seed):

        def create_cluster(train, test, features, kind, n_clusters):
            col_prefix = 'raw_clusters_'
            train_ = train[features].copy()
            test_ = test[features].copy()
            kmeans = KMeans(n_clusters=n_clusters, random_state=seed).fit(train_)
            data = pd.concat([train_, test_], axis=0).reset_index(drop=True)
            pred = pd.DataFrame(kmeans.predict(data[features]), columns=[col_prefix + kind])
            pred = pd.get_dummies(pred, columns=[col_prefix + kind])
            pred_train = pred.iloc[:len(train),:].reset_index(drop=True)
            pred_test = pred.iloc[len(train):,:].reset_index(drop=True)
            return pred_train, pred_test

        g_pred_train, g_pred_test = create_cluster(train, test, g_col, kind='g', n_clusters=n_clusters_g)
        c_pred_train, c_pred_test = create_cluster(train, test, c_col, kind='c', n_clusters=n_clusters_c)
        gc_pred_train = pd.concat([g_pred_train, c_pred_train], axis=1).reset_index(drop=True)
        gc_pred_test = pd.concat([g_pred_test, c_pred_test], axis=1).reset_index(drop=True)
        return gc_pred_train, gc_pred_test
    
    raw_gc_cluster_train, raw_gc_cluster_test = gc_cluster(x_train, x_test, n_clusters_g=n_clusters_g, n_clusters_c=n_clusters_c, seed=seed)
    print('cluster raw g/c', raw_gc_cluster_train.shape, raw_gc_cluster_test.shape)


    # function to cluster PCA g/c
    n_clusters = 5
    def fe_cluster_pca(train, test, n_clusters, seed):
        col_prefix = 'pca_clusters'
        kmeans = KMeans(n_clusters=n_clusters, random_state = seed).fit(train)
        data = pd.concat([train, test], axis=0).reset_index(drop=True)
        pred = pd.DataFrame(kmeans.predict(data), columns=[col_prefix])
        pred = pd.get_dummies(pred, columns=[col_prefix])
        pred_train = pred.iloc[:len(train),:].reset_index(drop=True)
        pred_test = pred.iloc[len(train):,:].reset_index(drop=True)
        return pred_train, pred_test
    pca_cluster_train, pca_cluster_test = fe_cluster_pca(pca_gc_train, pca_gc_test, n_clusters=n_clusters, seed=seed)
    print('cluster PCA g/c', pca_cluster_train.shape, pca_cluster_test.shape)


    # statistics features
    gsquarecols = ['g-574','g-211','g-216','g-0','g-255','g-577','g-153','g-389','g-60','g-370','g-248','g-167','g-203',
                   'g-177','g-301','g-332','g-517','g-6','g-744','g-224','g-162','g-3','g-736','g-486','g-283','g-22',
                   'g-359','g-361','g-440','g-335','g-106','g-307','g-745','g-146','g-416','g-298','g-666','g-91','g-17',
                   'g-549','g-145','g-157','g-768','g-568','g-396']
    def fe_stats(df):
        stat_df = dict()
        stat_df['g_sum'] = df[g_col].sum(axis = 1)
        stat_df['g_mean'] = df[g_col].mean(axis = 1)
        stat_df['g_std'] = df[g_col].std(axis = 1)
        stat_df['g_kurt'] = df[g_col].kurtosis(axis = 1)
        stat_df['g_skew'] = df[g_col].skew(axis = 1)
        stat_df['c_sum'] = df[c_col].sum(axis = 1)
        stat_df['c_mean'] = df[c_col].mean(axis = 1)
        stat_df['c_std'] = df[c_col].std(axis = 1)
        stat_df['c_kurt'] = df[c_col].kurtosis(axis = 1)
        stat_df['c_skew'] = df[c_col].skew(axis = 1)
        stat_df['gc_sum'] = df[g_col + c_col].sum(axis = 1)
        stat_df['gc_mean'] = df[g_col + c_col].mean(axis = 1)
        stat_df['gc_std'] = df[g_col + c_col].std(axis = 1)
        stat_df['gc_kurt'] = df[g_col + c_col].kurtosis(axis = 1)
        stat_df['gc_skew'] = df[g_col + c_col].skew(axis = 1)

        stat_df['c52_c42'] = df['c-52'] * df['c-42']
        stat_df['c13_c73'] = df['c-13'] * df['c-73']
        stat_df['c26_c13'] = df['c-23'] * df['c-13']
        stat_df['c33_c6'] = df['c-33'] * df['c-6']
        stat_df['c11_c55'] = df['c-11'] * df['c-55']
        stat_df['c38_c63'] = df['c-38'] * df['c-63']
        stat_df['c38_c94'] = df['c-38'] * df['c-94']
        stat_df['c13_c94'] = df['c-13'] * df['c-94']
        stat_df['c4_c52'] = df['c-4'] * df['c-52']
        stat_df['c4_c42'] = df['c-4'] * df['c-42']
        stat_df['c13_c38'] = df['c-13'] * df['c-38']
        stat_df['c55_c2'] = df['c-55'] * df['c-2']
        stat_df['c55_c4'] = df['c-55'] * df['c-4']
        stat_df['c4_c13'] = df['c-4'] * df['c-13']
        stat_df['c82_c42'] = df['c-82'] * df['c-42']
        stat_df['c66_c42'] = df['c-66'] * df['c-42']
        stat_df['c6_c38'] = df['c-6'] * df['c-38']
        stat_df['c2_c13'] = df['c-2'] * df['c-13']
        stat_df['c62_c42'] = df['c-62'] * df['c-42']
        stat_df['c90_c55'] = df['c-90'] * df['c-55']
        
        for feature in c_col:
            stat_df[f'{feature}_squared'] = df[feature] ** 2     
        for feature in gsquarecols:
            stat_df[f'{feature}_squared'] = df[feature] ** 2  
            
        stat_df = pd.DataFrame(stat_df)
        return stat_df

    stat_train, stat_test = fe_stats(x_train), fe_stats(x_test)
    print('statistics features', stat_train.shape, stat_test.shape)

    # combine all FE results
    x_train_fe = pd.concat([sig_train, cp_train, fs_train, raw_gc_cluster_train, pca_cluster_train, stat_train], axis=1).reset_index(drop=True)
    x_test_fe = pd.concat([sig_test, cp_test, fs_test, raw_gc_cluster_test, pca_cluster_test, stat_test], axis=1).reset_index(drop=True)
    print('combine all FE results', x_train_fe.shape, x_test_fe.shape)

    # remove ctrl in train and test
    x_train_fe = x_train_fe[x_train_fe.cp_type!='ctl_vehicle']
    x_test_fe = x_test_fe[x_test_fe.cp_type!='ctl_vehicle']
    x_train_fe = x_train_fe.drop('cp_type', axis=1).reset_index(drop=True)
    x_test_fe = x_test_fe.drop('cp_type', axis=1).reset_index(drop=True)
    print('remove ctrl in train and test', x_train_fe.shape, x_test_fe.shape)

    return x_train_fe, x_test_fe

In [6]:
# FE parameter estimation based on provided train and test
fe_data = train_features.append(train_features_extra).reset_index(drop=True)
_, x_train_fe = feature_engineering(fe_data, train_features, seed=42)
_, x_test_fe = feature_engineering(fe_data, test_features, seed=42)

sig_id & cp_type (27796, 2) (23814, 2)
OHE for cp_time, cp_dose (27796, 2) (23814, 2)
RankGauss scaling (27796, 872) (23814, 872)
     rg_g-0    rg_g-1    rg_g-2    rg_g-3    rg_g-4    rg_g-5    rg_g-6  \
0  1.146806  0.902075 -0.418339 -0.961202 -0.254770 -1.021300 -1.369236   
1  0.128824  0.676862  0.274345  0.090495  1.208863  0.688965  0.316734   
2  0.790372  0.939951  1.428097 -0.121817 -0.002067  1.495091  0.238763   
3 -0.729866 -0.277163 -0.441200  0.766612  2.347817 -0.862761 -2.308829   
4 -0.444558 -0.481202  0.974729  0.977467  1.468304 -0.874772 -0.372682   

     rg_g-7    rg_g-8    rg_g-9  ...   rg_c-90   rg_c-91   rg_c-92   rg_c-93  \
0 -0.029888  0.684319 -0.316668  ...  0.405455  0.362189  1.296097  0.830281   
1  0.556428 -0.539718  0.831972  ... -0.527074  1.127076  0.716060  0.047538   
2  0.363471 -0.003611  1.237966  ... -0.834469 -0.747431  0.952950  0.046551   
3  0.305225 -0.191898 -1.389591  ... -1.429097 -0.762287 -1.653318 -1.259768   
4 -0.212171 -1.0670

In [7]:
# join processed features, scored_targets, non-scored_targets, drug_ids
train = x_train_fe.merge(train_targets_scored, on='sig_id')
train = train.merge(train_targets_nonscored, on='sig_id')
train = train.merge(train_drug, on='sig_id')
test = x_test_fe.copy()

In [8]:
# define target column set
target_cols = [x for x in train_targets_scored.columns if x != 'sig_id']
aux_target_cols = [x for x in train_targets_nonscored.columns if x != 'sig_id']
all_target_cols = target_cols + aux_target_cols

num_targets = len(target_cols)
num_aux_targets = len(aux_target_cols)
num_all_targets = len(all_target_cols)

print('num_targets: {}'.format(num_targets))
print('num_aux_targets: {}'.format(num_aux_targets))
print('num_all_targets: {}'.format(num_all_targets))

num_targets: 206
num_aux_targets: 331
num_all_targets: 537


In [9]:
print(train.shape)
print(test.shape)
print(sample_submission.shape)

(21948, 1776)
(3624, 1238)
(3982, 207)


# Dataset Classes

In [10]:
# dataset classes
# remarks: for saving mamories

class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)
        }
        
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }

        return dct

In [11]:
# basic model train, validate, predict functions

def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()

        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    return final_loss

def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    return final_loss, valid_preds

# remarks: "inference" serves similar purpose as making prediction
def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    return preds

In [12]:
# label smoothing as a regularization technique

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1

        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
            
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

# Model 1 - NN with 4 Hidden Layers

In [13]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets):
        super(Model, self).__init__()
        self.hidden_size = [1500, 1250, 1000, 750]
        self.dropout_value = [0.5, 0.35, 0.3, 0.25]

        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.Linear(num_features, self.hidden_size[0])
        
        self.batch_norm2 = nn.BatchNorm1d(self.hidden_size[0])
        self.dropout2 = nn.Dropout(self.dropout_value[0])
        self.dense2 = nn.Linear(self.hidden_size[0], self.hidden_size[1])

        self.batch_norm3 = nn.BatchNorm1d(self.hidden_size[1])
        self.dropout3 = nn.Dropout(self.dropout_value[1])
        self.dense3 = nn.Linear(self.hidden_size[1], self.hidden_size[2])

        self.batch_norm4 = nn.BatchNorm1d(self.hidden_size[2])
        self.dropout4 = nn.Dropout(self.dropout_value[2])
        self.dense4 = nn.Linear(self.hidden_size[2], self.hidden_size[3])

        self.batch_norm5 = nn.BatchNorm1d(self.hidden_size[3])
        self.dropout5 = nn.Dropout(self.dropout_value[3])
        self.dense5 = nn.utils.weight_norm(nn.Linear(self.hidden_size[3], num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.leaky_relu(self.dense3(x))

        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = F.leaky_relu(self.dense4(x))

        x = self.batch_norm5(x)
        x = self.dropout5(x)
        x = self.dense5(x)
        return x

# remarks: this label smoothing class is never used later in the script
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)

        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
            
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))    

In [14]:
# class for Transfer Learning - create a new model by replacing last layer of the original model
# all parameters in old layers will be frozen

class FineTuneScheduler:
    def __init__(self, epochs):
        self.epochs = epochs
        self.epochs_per_step = 0
        self.frozen_layers = []

    def copy_without_top(self, model, num_features, num_targets, num_targets_new):
        self.frozen_layers = []

        model_new = Model(num_features, num_targets)
        model_new.load_state_dict(model.state_dict())

        # Freeze all weights
        for name, param in model_new.named_parameters():
            layer_index = name.split('.')[0][-1]

            if layer_index == 5:
                continue

            param.requires_grad = False

            # Save frozen layer names
            if layer_index not in self.frozen_layers:
                self.frozen_layers.append(layer_index)

        self.epochs_per_step = self.epochs // len(self.frozen_layers)

        # Replace the top layers with another ones
        model_new.batch_norm5 = nn.BatchNorm1d(model_new.hidden_size[3])
        model_new.dropout5 = nn.Dropout(model_new.dropout_value[3])
        model_new.dense5 = nn.utils.weight_norm(nn.Linear(model_new.hidden_size[-1], num_targets_new))
        model_new.to(DEVICE)
        return model_new

    def step(self, epoch, model):
        if len(self.frozen_layers) == 0:
            return

        if epoch % self.epochs_per_step == 0:
            last_frozen_index = self.frozen_layers[-1]
            
            # Unfreeze parameters of the last frozen layer
            for name, param in model.named_parameters():
                layer_index = name.split('.')[0][-1]

                if layer_index == last_frozen_index:
                    param.requires_grad = True

            del self.frozen_layers[-1]  # Remove the last layer as unfrozen

In [15]:
# function of basic One-Hot Encoding for cp featuers
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [16]:
# define number of features input to NN
feature_cols = [c for c in process_data(train).columns if c not in all_target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold', 'sig_id', 'drug_id']]
num_features = len(feature_cols)
num_features

1240

In [17]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 24
BATCH_SIZE = 128

WEIGHT_DECAY = {'ALL_TARGETS': 1e-5, 'SCORED_ONLY': 3e-6}
MAX_LR = {'ALL_TARGETS': 1e-2, 'SCORED_ONLY': 3e-3}
DIV_FACTOR = {'ALL_TARGETS': 1e3, 'SCORED_ONLY': 1e2}
PCT_START = 0.1

In [18]:
# Show model architecture
model = Model(num_features, num_all_targets)
model

Model(
  (batch_norm1): BatchNorm1d(1240, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense1): Linear(in_features=1240, out_features=1500, bias=True)
  (batch_norm2): BatchNorm1d(1500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (dense2): Linear(in_features=1500, out_features=1250, bias=True)
  (batch_norm3): BatchNorm1d(1250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout3): Dropout(p=0.35, inplace=False)
  (dense3): Linear(in_features=1250, out_features=1000, bias=True)
  (batch_norm4): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout4): Dropout(p=0.3, inplace=False)
  (dense4): Linear(in_features=1000, out_features=750, bias=True)
  (batch_norm5): BatchNorm1d(750, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout5): Dropout(p=0.25, inplace=False)
  (dense5): Linear(in_features=750, out_features=537, 

In [19]:
# stratified k-fold by drug_id

def make_cv_folds(train, SEEDS, NFOLDS, DRUG_THRESH):
    vc = train.drug_id.value_counts()
    vc1 = vc.loc[vc <= DRUG_THRESH].index.sort_values()
    vc2 = vc.loc[vc > DRUG_THRESH].index.sort_values()

    for seed_id in range(SEEDS):
        kfold_col = 'kfold_{}'.format(seed_id)
        
        # STRATIFY DRUGS 18X OR LESS
        dct1 = {}
        dct2 = {}

        skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed_id)
        tmp = train.groupby('drug_id')[target_cols].mean().loc[vc1]

        for fold,(idxT, idxV) in enumerate(skf.split(tmp, tmp[target_cols])):
            dd = {k: fold for k in tmp.index[idxV].values}
            dct1.update(dd)

        # STRATIFY DRUGS MORE THAN 18X
        skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed_id)
        tmp = train.loc[train.drug_id.isin(vc2)].reset_index(drop=True)

        for fold,(idxT, idxV) in enumerate(skf.split(tmp, tmp[target_cols])):
            dd = {k: fold for k in tmp.sig_id[idxV].values}
            dct2.update(dd)

        # ASSIGN FOLDS
        train[kfold_col] = train.drug_id.map(dct1)
        train.loc[train[kfold_col].isna(), kfold_col] = train.loc[train[kfold_col].isna(), 'sig_id'].map(dct2)
        train[kfold_col] = train[kfold_col].astype('int8')
        
    return train

SEEDS = 7
NFOLDS = 7
DRUG_THRESH = 18

train = make_cv_folds(train, SEEDS, NFOLDS, DRUG_THRESH)
train.head()

,sig_id,cp_dose,cp_time,rg_g-0,rg_g-1,rg_g-2,rg_g-3,rg_g-4,rg_g-5,rg_g-6,...,xanthine_oxidase_inhibitor,xiap_inhibitor,drug_id,kfold_0,kfold_1,kfold_2,kfold_3,kfold_4,kfold_5,kfold_6
0,id_000644bb2,0,0,1.146806,0.902075,-0.418339,-0.961202,-0.254770,-1.021300,-1.369236,...,0,0,b68db1d53,1,3,3,5,0,5,2
1,id_000779bfc,0,2,0.128824,0.676862,0.274345,0.090495,1.208863,0.688965,0.316734,...,0,0,df89a8e5a,0,3,6,3,3,4,1
2,id_000a6266a,0,1,0.790372,0.939951,1.428097,-0.121817,-0.002067,1.495091,0.238763,...,0,0,18bb41b2c,5,3,3,1,3,0,4
3,id_0015fd391,0,1,-0.729866,-0.277163,-0.441200,0.766612,2.347817,-0.862761,-2.308829,...,0,0,8c7f86626,4,3,1,2,4,5,2
4,id_001626bd3,1,2,-0.444558,-0.481202,0.974729,0.977467,1.468304,-0.874772,-0.372682,...,0,0,7cbed3131,6,5,3,3,2,1,4


In [20]:
# single-fold, single-seed training function

def run_training(fold_id, seed_id):
    seed_everything(seed_id)
    
    train_ = process_data(train)
    test_ = process_data(test)
    
    kfold_col = f'kfold_{seed_id}'
    trn_idx = train_[train_[kfold_col] != fold_id].index
    val_idx = train_[train_[kfold_col] == fold_id].index
    
    train_df = train_[train_[kfold_col] != fold_id].reset_index(drop=True)
    valid_df = train_[train_[kfold_col] == fold_id].reset_index(drop=True)
    
    def train_model(model, tag_name, target_cols_now, fine_tune_scheduler=None):
        x_train, y_train  = train_df[feature_cols].values, train_df[target_cols_now].values
        x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols_now].values
        
        train_dataset = MoADataset(x_train, y_train)
        valid_dataset = MoADataset(x_valid, y_valid)

        trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=WEIGHT_DECAY[tag_name])
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer,
                                                  steps_per_epoch=len(trainloader),
                                                  pct_start=PCT_START,
                                                  div_factor=DIV_FACTOR[tag_name], 
                                                  max_lr=MAX_LR[tag_name],
                                                  epochs=EPOCHS)
        
        loss_fn = nn.BCEWithLogitsLoss()
        loss_tr = SmoothBCEwLogits(smoothing=0.001)

        oof = np.zeros((len(train), len(target_cols_now)))
        best_loss = np.inf
        
        for epoch in range(EPOCHS):
            if fine_tune_scheduler is not None:
                fine_tune_scheduler.step(epoch, model)

            train_loss = train_fn(model, optimizer, scheduler, loss_tr, trainloader, DEVICE)
            valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
            print(f"SEED: {seed_id}, FOLD: {fold_id}, {tag_name}, EPOCH: {epoch}, train_loss: {train_loss:.6f}, valid_loss: {valid_loss:.6f}")

            if np.isnan(valid_loss):
                break
            
            if valid_loss < best_loss:
                best_loss = valid_loss
                oof[val_idx] = valid_preds
                torch.save(model.state_dict(), f"NN_{tag_name}_SEED{seed_id}_FOLD{fold_id}_.pth") # modified to save best model per seed and fold

        return oof

    fine_tune_scheduler = FineTuneScheduler(EPOCHS)

    pretrained_model = Model(num_features, num_all_targets)
    pretrained_model.to(DEVICE)

    # Train on scored + nonscored targets
    train_model(pretrained_model, 'ALL_TARGETS', all_target_cols)

    # Load the pretrained model with the best loss
    pretrained_model = Model(num_features, num_all_targets)
    pretrained_model.load_state_dict(torch.load(f"NN_ALL_TARGETS_SEED{seed_id}_FOLD{fold_id}_.pth"))
    pretrained_model.to(DEVICE)

    # Copy model without the top layer
    final_model = fine_tune_scheduler.copy_without_top(pretrained_model, num_features, num_all_targets, num_targets)

    # Fine-tune the model on scored targets only
    oof = train_model(final_model, 'SCORED_ONLY', target_cols, fine_tune_scheduler)

    # Load the fine-tuned model with the best loss
    model = Model(num_features, num_targets)
    model.load_state_dict(torch.load(f"NN_SCORED_ONLY_SEED{seed_id}_FOLD{fold_id}_.pth"))
    model.to(DEVICE)

    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    predictions = np.zeros((len(test_), num_targets))
    predictions = inference_fn(model, testloader, DEVICE)
    return oof, predictions

In [21]:
# single-seed training function

def run_k_fold(NFOLDS, seed_id):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold_id in range(NFOLDS):
        oof_, pred_ = run_training(fold_id, seed_id)
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [22]:
# training on multiple SEEDS

SEED = [0, 1, 2, 3, 4, 5, 6]
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

time_begin = time()

for seed_id in SEED:
    oof_, predictions_ = run_k_fold(NFOLDS, seed_id)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = predictions

# report total training time
time_diff = time() - time_begin
print(str(timedelta(seconds=time_diff)))

SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 0, train_loss: 0.495827, valid_loss: 0.019534
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 1, train_loss: 0.016943, valid_loss: 0.010596
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 2, train_loss: 0.014879, valid_loss: 0.010201
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 3, train_loss: 0.014007, valid_loss: 0.010136
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 4, train_loss: 0.014110, valid_loss: 0.010138
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 5, train_loss: 0.013947, valid_loss: 0.010213
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 6, train_loss: 0.013973, valid_loss: 0.010229
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 7, train_loss: 0.013985, valid_loss: 0.010278
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 8, train_loss: 0.013960, valid_loss: 0.010132
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 9, train_loss: 0.013926, valid_loss: 0.010111
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 10, train_loss: 0.013914, valid_loss: 0.010172
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 11, train_loss: 0.013871, valid_loss:

In [23]:
# obtain CV score

valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0) # remarks: here already set 0 for ctrl 

y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0

for i in range(len(target_cols)):
    score += log_loss(y_true[:, i], y_pred[:, i])

print("CV log_loss: ", score / y_pred.shape[1])

CV log_loss:  0.015603867671285977


# Submission

In [24]:
# create submission file
sub1 = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub1.to_csv('/kaggle/working/submission.csv', index = None)
print(sub1.shape)
sub1.head()

(3982, 207)


,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.000902,0.000955,0.002047,0.017687,0.021500,0.004837,0.003854,0.005194,0.000397,...,0.001267,0.000958,0.004336,0.001701,0.001320,0.000593,0.001325,0.002217,0.002516,0.001712
1,id_001897cda,0.000626,0.000829,0.001074,0.002075,0.001293,0.001704,0.003664,0.014857,0.026774,...,0.000581,0.000903,0.001758,0.000693,0.010186,0.000667,0.005990,0.001720,0.001191,0.003740
2,id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,id_00276f245,0.000744,0.000865,0.001587,0.011485,0.020147,0.004411,0.003006,0.005945,0.000454,...,0.000594,0.001210,0.002089,0.003233,0.004167,0.000535,0.001421,0.001841,0.001978,0.001835
4,id_0027f1083,0.002174,0.001318,0.001602,0.020017,0.025909,0.005222,0.003786,0.002311,0.000471,...,0.000888,0.000812,0.002916,0.001612,0.001579,0.000635,0.001098,0.002022,0.000831,0.001618
